# Repated

In [108]:
!pip install bambi

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import pymc as pm
import arviz as az
import bambi as bmb  # For easier Bayesian regression models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import warnings
from sklearn.metrics import roc_curve, auc


In [110]:
warnings.filterwarnings('ignore')  # Suppress warnings
np.random.seed(42)  # For reproducibility


In [111]:
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)


In [112]:
df = pd.read_csv('./weatherAUS.csv')
df


,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145455,2017-06-21,Uluru,2.8,23.4,0.0,NaN,NaN,E,31.0,SE,ENE,13.0,11.0,51.0,24.0,1024.6,1020.3,NaN,NaN,10.1,22.4,No,No
145456,2017-06-22,Uluru,3.6,25.3,0.0,NaN,NaN,NNW,22.0,SE,N,13.0,9.0,56.0,21.0,1023.5,1019.1,NaN,NaN,10.9,24.5,No,No
145457,2017-06-23,Uluru,5.4,26.9,0.0,NaN,NaN,N,37.0,SE,WNW,9.0,9.0,53.0,24.0,1021.0,1016.8,NaN,NaN,12.5,26.1,No,No
145458,2017-06-24,Uluru,7.8,27.0,0.0,NaN,NaN,SE,28.0,SSE,N,13.0,7.0,51.0,24.0,1019.4,1016.5,3.0,2.0,15.1,26.0,No,No


In [113]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   Evaporation    82670 non-null   float64
 6   Sunshine       75625 non-null   float64
 7   WindGustDir    135134 non-null  object 
 8   WindGustSpeed  135197 non-null  float64
 9   WindDir9am     134894 non-null  object 
 10  WindDir3pm     141232 non-null  object 
 11  WindSpeed9am   143693 non-null  float64
 12  WindSpeed3pm   142398 non-null  float64
 13  Humidity9am    142806 non-null  float64
 14  Humidity3pm    140953 non-null  float64
 15  Pressure9am    130395 non-null  float64
 16  Pressure3pm    130432 non-null  float64
 17  Cloud9am       89572 non-null

In [114]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
MinTemp,143975.0,12.194034,6.398495,-8.5,7.6,12.0,16.9,33.9
MaxTemp,144199.0,23.221348,7.119049,-4.8,17.9,22.6,28.2,48.1
Rainfall,142199.0,2.360918,8.478060,0.0,0.0,0.0,0.8,371.0
Evaporation,82670.0,5.468232,4.193704,0.0,2.6,4.8,7.4,145.0
Sunshine,75625.0,7.611178,3.785483,0.0,4.8,8.4,10.6,14.5
WindGustSpeed,135197.0,40.035230,13.607062,6.0,31.0,39.0,48.0,135.0
WindSpeed9am,143693.0,14.043426,8.915375,0.0,7.0,13.0,19.0,130.0
WindSpeed3pm,142398.0,18.662657,8.809800,0.0,13.0,19.0,24.0,87.0
Humidity9am,142806.0,68.880831,19.029164,0.0,57.0,70.0,83.0,100.0
Humidity3pm,140953.0,51.539116,20.795902,0.0,37.0,52.0,66.0,100.0


In [115]:
missing_values = df.isna().sum()
print(missing_values)
print(f"Missing values percentage: {missing_values.sum() / (df.shape[0] * df.shape[1]):.2%}")


Date                 0
Location             0
MinTemp           1485
MaxTemp           1261
Rainfall          3261
Evaporation      62790
Sunshine         69835
WindGustDir      10326
WindGustSpeed    10263
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am      1767
WindSpeed3pm      3062
Humidity9am       2654
Humidity3pm       4507
Pressure9am      15065
Pressure3pm      15028
Cloud9am         55888
Cloud3pm         59358
Temp9am           1767
Temp3pm           3609
RainToday         3261
RainTomorrow      3267
dtype: int64
Missing values percentage: 10.26%


In [116]:
df.columns

Index(['Date', 'Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation',
       'Sunshine', 'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm',
       'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am',
       'Temp3pm', 'RainToday', 'RainTomorrow'],
      dtype='object')

In [117]:
df.dtypes

,0
Date,object
Location,object
MinTemp,float64
MaxTemp,float64
Rainfall,float64
Evaporation,float64
Sunshine,float64
WindGustDir,object
WindGustSpeed,float64
WindDir9am,object


In [118]:
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['Season'] = pd.cut(df['Month'],
                      bins=[0, 3, 6, 9, 12],
                      labels=['Summer', 'Autumn', 'Winter', 'Spring'],
                      include_lowest=True)  # Southern hemisphere seasons

print("Date features added.")

Date features added.


In [119]:
df['RainToday'] = df['RainToday'].map({'Yes': 1, 'No': 0})
df['RainTomorrow'] = df['RainTomorrow'].map({'Yes': 1, 'No': 0})


In [120]:
df_clean = df.dropna()

In [121]:
df_clean

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,Year,Month,Day,Season
6049,2009-01-01,Cobar,17.9,35.2,0.0,12.0,12.3,SSW,48.0,ENE,SW,6.0,20.0,20.0,13.0,1006.3,1004.4,2.0,5.0,26.6,33.4,0.0,0.0,2009,1,1,Summer
6050,2009-01-02,Cobar,18.4,28.9,0.0,14.8,13.0,S,37.0,SSE,SSE,19.0,19.0,30.0,8.0,1012.9,1012.1,1.0,1.0,20.3,27.0,0.0,0.0,2009,1,2,Summer
6052,2009-01-04,Cobar,19.4,37.6,0.0,10.8,10.6,NNE,46.0,NNE,NNW,30.0,15.0,42.0,22.0,1012.3,1009.2,1.0,6.0,28.7,34.9,0.0,0.0,2009,1,4,Summer
6053,2009-01-05,Cobar,21.9,38.4,0.0,11.4,12.2,WNW,31.0,WNW,WSW,6.0,6.0,37.0,22.0,1012.7,1009.1,1.0,5.0,29.1,35.6,0.0,0.0,2009,1,5,Summer
6054,2009-01-06,Cobar,24.2,41.0,0.0,11.2,8.4,WNW,35.0,NW,WNW,17.0,13.0,19.0,15.0,1010.7,1007.4,1.0,6.0,33.6,37.6,0.0,0.0,2009,1,6,Summer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142298,2017-06-20,Darwin,19.3,33.4,0.0,6.0,11.0,ENE,35.0,SE,NE,9.0,20.0,63.0,32.0,1013.9,1010.5,0.0,1.0,24.5,32.3,0.0,0.0,2017,6,20,Autumn
142299,2017-06-21,Darwin,21.2,32.6,0.0,7.6,8.6,E,37.0,SE,SE,13.0,11.0,56.0,28.0,1014.6,1011.2,7.0,0.0,24.8,32.0,0.0,0.0,2017,6,21,Autumn
142300,2017-06-22,Darwin,20.7,32.8,0.0,5.6,11.0,E,33.0,E,W,17.0,11.0,46.0,23.0,1015.3,1011.8,0.0,0.0,24.8,32.1,0.0,0.0,2017,6,22,Autumn
142301,2017-06-23,Darwin,19.5,31.8,0.0,6.2,10.6,ESE,26.0,SE,NNW,9.0,17.0,62.0,58.0,1014.9,1010.7,1.0,1.0,24.8,29.2,0.0,0.0,2017,6,23,Autumn


In [122]:
df_clean.dtypes

,0
Date,datetime64[ns]
Location,object
MinTemp,float64
MaxTemp,float64
Rainfall,float64
Evaporation,float64
Sunshine,float64
WindGustDir,object
WindGustSpeed,float64
WindDir9am,object


# Bayesian

## Normal Bayesian Dist 


### Prior distribution: our belief about the temperature before seeing data (Normal)

### Likelihood: The data you observe 

### Posterior distribution: Updated belief after observing each new data point (Normal)

#### this part applies Bayesian inference to sequentially update our belief about the mean of the maximum temperature in a city Sydney or any other city we want using observed data over time

In [ ]:
# this func calc the postrior given the prior and only one data point
def bayesian_normal_update(prior_mean, prior_var, data, data_var):
    """
    Bayesian updating with normal prior and normal likelihood.

    Parameters:
    - prior_mean: Mean of prior which is normal distribution
    - prior_var: Variance of prior which is normal distribution
    - data: the data :) 
    - data_var: Variance of likelihood function

    Returns:
    - posterior_mean: Updated mean
    - posterior_var: Updated variance
    """
    # Precision 
    prior_precision = 1 / prior_var
    data_precision = 1 / data_var

    # Calculate posterior precision and mean
    posterior_precision = prior_precision + data_precision
    posterior_var = 1 / posterior_precision
    posterior_mean = (prior_mean * prior_precision + data * data_precision) / posterior_precision

    return posterior_mean, posterior_var

# sequential updating for any location

location = 'Sydney'
sydney_data = df_clean[df_clean['Location'] == location].copy()
sydney_data = sydney_data.sort_values('Date')

# we will track the sequential updates of our belief about mean max temperature

# prior belief 
prior_mean = 20  # initial guess for max temperature in Sydney 
prior_var = 100  

#  to store posterior updates
posteriors = []
dates = []

# data variance 
data_var = sydney_data['MaxTemp'].var()

# sequential updating row by row
for i, (_, row) in enumerate(sydney_data.iterrows()):
    # Get data point
    data = row['MaxTemp']

    # update posterior
    posterior_mean, posterior_var = bayesian_normal_update(prior_mean, prior_var, data, data_var)

    # Store posterior parameters
    posteriors.append((posterior_mean, posterior_var))
    dates.append(row['Date'])

    # Update prior for next iteration
    prior_mean, prior_var = posterior_mean, posterior_var

    # Print update every 100 data points
    if i % 100 == 0:
        print(f"After {i} observations, posterior mean = {posterior_mean:.2f}°C, posterior std = {np.sqrt(posterior_var):.2f}")

# Extract final posterior parameters
final_mean, final_var = posteriors[-1]
print(f"\nFinal posterior for {location} MaxTemp:")
print(f"Mean = {final_mean:.2f}°C, Std = {np.sqrt(final_var):.2f}")

# Plot the evolution of the posterior mean and credible interval
means = [p[0] for p in posteriors]
stds = [np.sqrt(p[1]) for p in posteriors]

plt.figure(figsize=(14, 8))
plt.plot(dates, means, 'b-', label='Posterior Mean')
plt.fill_between(dates,
                 [m - 2*s for m, s in zip(means, stds)],
                 [m + 2*s for m, s in zip(means, stds)],
                 color='b', alpha=0.2, label='95% Credible Interval')
plt.xlabel('Date')
plt.ylabel('MaxTemp (°C)')
plt.title(f'Sequential Bayesian Updating of Mean MaxTemp in {location}')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('bayesian_updating.png')
plt.close()


After 0 observations, posterior mean = 20.25°C, posterior std = 4.09
After 100 observations, posterior mean = 21.34°C, posterior std = 0.45
After 200 observations, posterior mean = 22.90°C, posterior std = 0.32
After 300 observations, posterior mean = 23.17°C, posterior std = 0.26
After 400 observations, posterior mean = 22.43°C, posterior std = 0.22
After 500 observations, posterior mean = 23.07°C, posterior std = 0.20
After 600 observations, posterior mean = 23.54°C, posterior std = 0.18
After 700 observations, posterior mean = 23.16°C, posterior std = 0.17
After 800 observations, posterior mean = 23.00°C, posterior std = 0.16
After 900 observations, posterior mean = 23.44°C, posterior std = 0.15
After 1000 observations, posterior mean = 23.39°C, posterior std = 0.14
After 1100 observations, posterior mean = 22.99°C, posterior std = 0.13
After 1200 observations, posterior mean = 23.20°C, posterior std = 0.13
After 1300 observations, posterior mean = 23.48°C, posterior std = 0.12
Afte

## Linear Bayesian Regression

In [ ]:
X = df_clean[['MinTemp', 'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm']]
y = df_clean['MaxTemp']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

with pm.Model() as blr_model:
    # prior of the bias :)
    intercept = pm.Normal('intercept', mu=0, sigma=10)

    # prior for the betas :) 
    beta = pm.Normal('beta', mu=0, sigma=1, shape=X_train_scaled.shape[1])

    # The predicted mean (mu) is the dot product of the inputs and coefficients + the intercept
    # μ = Bo + B1x1 + B2x2 + .......
    mu = intercept + pm.math.dot(X_train_scaled, beta)

    # prior of the std 
    sigma = pm.HalfNormal('sigma', sigma=1)

    # likelihood and it is taken from the training data ;)
    likelihood = pm.Normal('likelihood', mu=mu, sigma=sigma, observed=y_train)


    # Sample from the posterior / it uses markov chain monte carlo to draw samples from the postirior dist  :)
    # MCMC lets us approximate the posterior by:
    # Generating a chain of samples that converge to the true posterior distribution.
    # These samples are then used to:
    # Estimate the mean, std, CI 
    # Make predictions
    # Each sample is one set of values for: intercept All elements of beta and std 
    trace_blr = pm.sample(1000, tune=1000, cores=2, random_seed=42)


# Results :)
print("Bayesian Linear Regression Summary:")
summary = az.summary(trace_blr, var_names=['intercept', 'beta', 'sigma'])
print(summary)

# Get posterior samples
posterior_samples = trace_blr.posterior

# Calculate predictions on the test set
n_samples = len(posterior_samples['intercept'][0])
y_pred_samples = np.zeros((len(X_test), n_samples))

for i in range(n_samples): # for each posterior sample
    intercept_sample = posterior_samples['intercept'][0, i].item() # extract sampled intercept 

    beta_samples = np.array([posterior_samples['beta'][0, i, j].item() for j in range(X_test_scaled.shape[1])]) # and coefficients
    y_pred_samples[:, i] = intercept_sample + np.dot(X_test_scaled, beta_samples) # predict all test samples :0

# calc mean and 95% ci 
y_pred_mean = np.mean(y_pred_samples, axis=1)
y_pred_lower = np.percentile(y_pred_samples, 2.5, axis=1)
y_pred_upper = np.percentile(y_pred_samples, 97.5, axis=1)

# calc RMSE
rmse = np.sqrt(np.mean((y_pred_mean - y_test) ** 2))
print(f"\nRoot Mean Square Error (RMSE): {rmse:.2f}")

# Plot predictions vs actuals
plt.figure(figsize=(12, 8))
plt.scatter(y_test, y_pred_mean, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--')
plt.xlabel('Actual MaxTemp')
plt.ylabel('Predicted MaxTemp')
plt.title('Bayesian Linear Regression: Predicted vs Actual MaxTemp')
plt.tight_layout()
plt.savefig('blr_predictions.png')
plt.close()

# Plot posterior distributions of coefficients
feature_names = ['MinTemp', 'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm']
az.plot_posterior(trace_blr, var_names="beta", figsize=(14, 10))
plt.title('Posterior Distributions of Regression Coefficients')
plt.tight_layout()
plt.savefig('blr_coefficients.png')
plt.close()


Output()


Bayesian Linear Regression Summary:
             mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  \
intercept  24.215  0.013  24.191   24.242      0.000    0.000    2385.0   
beta[0]     5.137  0.016   5.107    5.165      0.000    0.000    2144.0   
beta[1]    -0.229  0.019  -0.265   -0.194      0.001    0.000    1444.0   
beta[2]    -2.975  0.019  -3.011   -2.939      0.001    0.000    1398.0   
beta[3]     4.073  0.051   3.982    4.171      0.002    0.001    1106.0   
beta[4]    -4.417  0.052  -4.516   -4.322      0.002    0.001    1133.0   
sigma       2.778  0.009   2.761    2.794      0.000    0.000    2192.0   

           ess_tail  r_hat  
intercept    1299.0    1.0  
beta[0]      1395.0    1.0  
beta[1]      1203.0    1.0  
beta[2]      1442.0    1.0  
beta[3]      1120.0    1.0  
beta[4]      1181.0    1.0  
sigma        1552.0    1.0  

Root Mean Square Error (RMSE): 2.77


## Bayesian Logistic Regression

In [ ]:
X_rain = df_clean[['MaxTemp', 'Humidity3pm', 'Pressure3pm', 'RainToday']].copy()
y_rain = df_clean['RainTomorrow'].copy()

X_train_rain, X_test_rain, y_train_rain, y_test_rain = train_test_split(
    X_rain, y_rain, test_size=0.2, random_state=42, stratify=y_rain
)


scaler_rain = StandardScaler()
X_train_rain_scaled = scaler_rain.fit_transform(X_train_rain)
X_test_rain_scaled = scaler_rain.transform(X_test_rain)

with pm.Model() as logistic_model:

    # priors for regression coefficients
    intercept = pm.Normal('intercept', mu=0, sigma=10)
    beta = pm.Normal('beta', mu=0, sigma=1, shape=X_train_rain_scaled.shape[1])

    # Expected value 
    log_odds = intercept + pm.math.dot(X_train_rain_scaled, beta)

    # 1 / e^-z
    p = pm.math.invlogit(log_odds)

    # likelihood (Bernoulli) bec it is 0 or 1  p or q :) 
    likelihood = pm.Bernoulli('likelihood', p=p, observed=y_train_rain)

    # Sample from the posterior
    trace_logistic = pm.sample(1000, tune=1000, cores=2, random_seed=42)

# Results
print("Bayesian Logistic Regression Summary:") 
summary_logistic = az.summary(trace_logistic, var_names=['intercept', 'beta'])
print(summary_logistic)

# calc predictions
n_samples = len(trace_logistic.posterior['intercept'][0])
p_pred_samples = np.zeros((len(X_test_rain), n_samples))

for i in range(n_samples):
    intercept_sample = trace_logistic.posterior['intercept'][0, i].item()
    beta_samples = np.array([trace_logistic.posterior['beta'][0, i, j].item() for j in range(X_test_rain_scaled.shape[1])])
    log_odds = intercept_sample + np.dot(X_test_rain_scaled, beta_samples)
    p_pred_samples[:, i] = 1 / (1 + np.exp(-log_odds))

# calc mean predictions and 95% ci
p_pred_mean = np.mean(p_pred_samples, axis=1)
p_pred_lower = np.percentile(p_pred_samples, 2.5, axis=1)
p_pred_upper = np.percentile(p_pred_samples, 97.5, axis=1)

# convert probabilities to class labels      0.5 our threshold :)
y_pred_class = (p_pred_mean > 0.5).astype(int)

# accuracy
accuracy = np.mean(y_pred_class == y_test_rain)
print(f"Accuracy: {accuracy:.2f}")

# confusion matrix
conf_matrix = pd.crosstab(y_test_rain, y_pred_class,
                         rownames=['Actual'], colnames=['Predicted'])
print("Confusion Matrix:")
print(conf_matrix)

# calc precision, recall, and F1 score
true_positives = conf_matrix.loc[1, 1]
false_positives = conf_matrix.loc[0, 1]
false_negatives = conf_matrix.loc[1, 0]

precision = true_positives / (true_positives + false_positives)
recall = true_positives / (true_positives + false_negatives)
f1_score = 2 * (precision * recall) / (precision + recall)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1_score:.2f}")

# Create a plot of coefficient posterior distributions
feature_names_rain = ['MaxTemp', 'Humidity3pm', 'Pressure3pm', 'RainToday']
az.plot_posterior(trace_logistic, var_names="beta", figsize=(14, 10))
plt.title('Posterior Distributions of Logistic Regression Coefficients')
plt.tight_layout()
plt.savefig('logistic_coefficients.png')
plt.close()


fpr, tpr, thresholds = roc_curve(y_test_rain, p_pred_mean)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(10, 8))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.tight_layout()
plt.savefig('roc_curve.png')
plt.close()

Output()


Bayesian Logistic Regression Summary:
            mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  \
intercept -1.827  0.017  -1.857   -1.793        0.0      0.0    1725.0   
beta[0]   -0.247  0.017  -0.281   -0.217        0.0      0.0    1881.0   
beta[1]    1.288  0.018   1.253    1.321        0.0      0.0    1766.0   
beta[2]   -0.741  0.017  -0.773   -0.711        0.0      0.0    1963.0   
beta[3]    0.248  0.013   0.226    0.273        0.0      0.0    1955.0   

           ess_tail  r_hat  
intercept    1728.0    1.0  
beta[0]      1647.0    1.0  
beta[1]      1600.0    1.0  
beta[2]      1686.0    1.0  
beta[3]      1630.0    1.0  

Accuracy: 0.84

Confusion Matrix:
Predicted     0     1
Actual               
0.0        8366   433
1.0        1332  1153

Precision: 0.73
Recall: 0.46
F1 Score: 0.57


## Change Point Detection

In [ ]:
df_clean['YearMonth'] = df_clean['Date'].dt.to_period('M') # convert each date to monthly period like 2020-01-15 → 2020-01
monthly_temp = df_clean.groupby('YearMonth')['MaxTemp'].mean().reset_index() # group data by each month and calculates the mean maximum temperature for each month
monthly_temp['YearMonth'] = monthly_temp['YearMonth'].dt.to_timestamp() 
monthly_temp = monthly_temp.sort_values('YearMonth')

# convert to arrays
dates = monthly_temp['YearMonth'].values
temps = monthly_temp['MaxTemp'].values
n_data = len(temps)

with pm.Model() as change_point_model:
    # prior for change point location
    change_point = pm.DiscreteUniform('change_point', lower=0, upper=n_data-1) # i chose discrete uniform because it is all equally likely in the begining :) 

    # prior for mean values before and after change point
    mean_before = pm.Normal('mean_before', mu=20, sigma=5)
    mean_after = pm.Normal('mean_after', mu=20, sigma=5)

    # prior for precision 
    precision = pm.Gamma('precision', alpha=2, beta=1)
    sigma = pm.Deterministic('sigma', 1.0 / pm.math.sqrt(precision))

    # likelihood
    idx = np.arange(n_data)
    mu = pm.math.switch(idx <= change_point, mean_before, mean_after)
    likelihood = pm.Normal('likelihood', mu=mu, sigma=sigma, observed=temps)


    # sample from the posterior
    trace_cp = pm.sample(1000, tune=1000, cores=2, random_seed=42) # each sample has change_point, mean_before, mean_after, std

# results
print("Change Point Detection Summary:")
print(az.summary(trace_cp, var_names=['change_point', 'mean_before', 'mean_after', 'sigma']))

# Extract change point
cp_samples = trace_cp.posterior['change_point'].values.flatten()
cp_mode = int(stats.mode(cp_samples)[0])
cp_date = dates[cp_mode]
print(f"Most probable change point: {cp_date}")

# plot the change point model
plt.figure(figsize=(14, 8))
plt.plot(dates, temps, 'o', alpha=0.5)

# plot the mean values before and after the change point
mean_before_samples = trace_cp.posterior['mean_before'].values.flatten()
mean_after_samples = trace_cp.posterior['mean_after'].values.flatten()

mean_before_avg = np.mean(mean_before_samples)
mean_after_avg = np.mean(mean_after_samples)

plt.axvline(cp_date, color='r', linestyle='--', alpha=0.5, label=f'Change Point: {cp_date}')
plt.axhline(mean_before_avg, color='g', linestyle='-', alpha=0.5,
            label=f'Mean Before: {mean_before_avg:.2f}°C')
plt.axhline(mean_after_avg, color='b', linestyle='-', alpha=0.5,
            label=f'Mean After: {mean_after_avg:.2f}°C')

plt.xlabel('Date')
plt.ylabel('Average Maximum Temperature (°C)')
plt.title('Bayesian Change Point Detection for Monthly Average Maximum Temperature')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('change_point.png')
plt.close()

Output()

ERROR:pymc.stats.convergence:There were 257 divergences after tuning. Increase `target_accept` or reparameterize.
ERROR:pymc.stats.convergence:The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details



Change Point Detection Summary:
                mean      sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  \
change_point  53.622  39.735   0.000  110.000     11.479    1.512      15.0   
mean_before   23.296   1.441  20.215   25.076      0.324    0.230      27.0   
mean_after    23.938   1.811  20.853   27.884      0.136    0.307     256.0   
sigma          4.247   0.279   3.691    4.751      0.034    0.025      70.0   

              ess_tail  r_hat  
change_point      86.0   1.13  
mean_before       48.0   1.08  
mean_after       113.0   1.11  
sigma             68.0   1.04  

Most probable change point: 2008-09-01T00:00:00.000000000
